In [ ]:
# Import libraries
import requests
from bs4 import BeautifulSoup

In [ ]:

URL = "https://www.gafeo.fr/admin/tool/reportsessions/index.php"#"http://annuairesante.ameli.fr/"
idToFind= "maincontent"
className="id_individuel"
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())

results = soup.find(id=idToFind)
results_class = soup.find_all("option")
print(results)




In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

formation_recherche = "INF002"
# async def main():
options = Options()
options.binary_location = "/Applications/Internet/Google Chrome.app/Contents/MacOS/Google Chrome"

PATH="/Applications/chromedriver"

browser = webdriver.Chrome(options= options, executable_path=PATH)
browser.get("https://www.gafeo.fr/admin/tool/reportsessions/index.php")
browser.find_element(By.ID,'username').send_keys("andria.c@eronservice.fr")
browser.find_element(By.ID,'password').send_keys("Andria-2021!")
browser.find_element(By.CSS_SELECTOR,"button.btn-primary").click()

pdf_downloaded=0
try:
        formation_session_dropdown =  Select(WebDriverWait(browser,10).until(EC.presence_of_element_located((By.CLASS_NAME,"formation_session"))))
        all_Options_Formation = formation_session_dropdown.options
        options_formations = [ option.text for option in all_Options_Formation if formation_recherche in option.text ]

        formation_session_dropdown.select_by_visible_text(options_formations[1])
        
        group_session_dropdown = Select(WebDriverWait(browser,10).until(EC.presence_of_element_located((By.CLASS_NAME,"group_session"))))
        all_options_gp_formation = group_session_dropdown.options
        options_gp_formations = [ option.text for option in all_options_gp_formation if formation_recherche in option.text ]

        
        group_session_dropdown.select_by_visible_text(options_gp_formations[1])

        contenu=WebDriverWait(browser,10).until(EC.presence_of_element_located((By.CLASS_NAME,'contenu')))
        elems = WebDriverWait(contenu,10).until(EC.presence_of_all_elements_located((By.TAG_NAME,'a')))
        for elem in  elems :
                WebDriverWait(browser,10).until(EC.presence_of_element_located((By.LINK_TEXT,elem.accessible_name))).click()
        
        print("Nombre d'apprenant:"+str(len(elems)))
        while pdf_downloaded !=len(elems):
                find_href_pdf_elems = WebDriverWait(browser,10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,'.contenu li div a')))
                print("Nombre d'éléments téléchargeable trouvé:" +str(len(find_href_pdf_elems)))
                if len(find_href_pdf_elems)==0:
                        continue
                else:
                        i=0
                        for el in find_href_pdf_elems:
                                print("Nombre de fichier téléchargé avant if :"+ str(pdf_downloaded))
                                # browser.execute_script("arguments[0].click();", el) fonctionne aussi
                                if (i<=pdf_downloaded-1 and pdf_downloaded !=0 ):
                                        i=i+1
                                        print("comparison between i: "+ str(i) + " and nb pdf" + str(pdf_downloaded))
                                        continue
                                else:
                                        print("Boucle dans la liste des éléments avec href:"+str(i))
                                        el.send_keys(Keys.RETURN)
                                        pdf_downloaded=pdf_downloaded+1
                                        print("Nombre de fichier téléchargé: "+ str(pdf_downloaded) + "/"+str(len(elems)))
                                i=i+1
             
except NoSuchElementException:
    print("There was an error, no such element")
else:
    print("There were no errors.")
finally:
    print("Process completed.")
        # browser.quit()

# await asyncio.sleep(2) 
# elements[42].click() WORKS donwload csv
# print(WebDriverWait(browser, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "li.list-group-item  > a[href]"))).get_attribute("href"))
# await main()


/var/folders/9g/9qptqtl550vchpg029w9bzjr0000gn/T/ipykernel_10003/1754461490.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(options= options, executable_path=PATH)


Process completed.


KeyboardInterrupt: 